# <span style="color:red">Before you turn this in, make sure everything runs as expected.</span>

1. **RESTART THE KERNEL** – in the menubar, select Kernel$\rightarrow$Restart
2. **RUN ALL CELLS** – in the menubar, select Cell$\rightarrow$Run All
3. **VALIDATE THE NOTEBOOK** – in the menubar, click the Validate button

## <span style="color:blue">How to Answer Questions</span>

### <span style="color:blue">Python code answers</span>

Enter your answer any place that says
```python
# Enter your code here
```
<span style="color:red">**AND delete the text.**</span>
```python
raise NotImplementedError # No Answer - remove if you provide an answer
```

### <span style="color:blue">Written answers</span>

Enter your answer any place that says
```
YOUR ANSWER HERE.
```

In [34]:
ANUID = "u7522927"

# Part 2 -- Sequence comparison assignment

This notebook is worth 14%.

There are extension questions in this notebook worth 2%. (See the [README-FIRST](README-FIRST.ipynb) notebook to understand how these work.)

Unless stated otherwise, <span style="color:red"><b>for non-code answers ≥50 words</b> answer in the style of the logic assignment (you do not have to list assumptions.)</span>

**You are expected to use Pythons builtins, numpy and plotly. You are not allowed to use anything else.**

## Background

We are interested in understanding how Transcription Factor (henceforth TF) binding information is encoded and how multiple TF's work together to affect gene expression. We are explicitly interested in asking ask whether the TF's we consider for this assignment (see below and the [data notebook](data_description.ipynb)) combine via cis-regulatory modules (CRMs) to affect development of heart cells.

The steps to addressing that question first require assessing whether there are patterns in the occurrence of the TFBS and, with reference to the [Culley et al](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3684448/) paper and the background material, design a computational experiment to evaluate a hypothesis you specify about CRM's.

There are 7 position weight matrices (PWM) downloaded from jaspar. (See the [data notebook](data_description.ipynb) for more details.) For each PWM, I have already computed the scores for all ~20k mouse genes for each position of the sequences. I demonstrate using that data below.

*For the final question in this notebook*, there is a also a file containing the gene expression measurements from the type of cell of interest. (See the [data notebook](data_description.ipynb) for more details.)

In [35]:
import glob

# listing all the jaspar TFBS pwm files
glob.glob("data/pssm/*.jaspar")

['data/pssm/srf1.jaspar',
 'data/pssm/tbx5.jaspar',
 'data/pssm/tbp.jaspar',
 'data/pssm/nkx2-5.jaspar',
 'data/pssm/mef2.jaspar',
 'data/pssm/myog.jaspar',
 'data/pssm/gata4.jaspar']

## Demonstration of a spatial pattern in TFBS occurrence

- Done for TBP (TATA binding protein)
- In this case *I chose an arbitrary value* for my cutoff as 7.
- Feel free to copy and modify my code for your answers

In [36]:
import plotly.express as px

In [37]:
import numpy

# loading a numpy array from file
tata_scores = numpy.load("data/pssm_scores/tbp.npy")
# how many positions had a value ≥ cutoff
tata_gt7 = (tata_scores > 7).sum(axis=0)
# mid-point=TSS
TSS = tata_gt7.shape[0] // 2
# produce x-axis labels as a number series corresponding to relative distance to TSS
x_axis = [i - TSS for i in range(tata_gt7.shape[0])]

In [38]:
fig = px.line(
    x=x_axis,
    y=tata_gt7,
    line_shape="spline",
    title="TATA Binding Protein PSSM matches",
    labels=dict(x="Position relative to TSS", y="Number of Genes Matching"),
)
fig.show()

## Q1 -- interpretation of TBP location

In ≤ 100 words (or a single word!) for each part, answer the following

- define what would a random distribution of matches look like
- is the distribution of matches to TBP shown above random according to your definition?

A random distribution of matches would be expected to have an approximately uniform number of matches across all positions relative to the TSS. 

The distribution of matches to TBP shown above does not appear to be random, as there is a clear peak in the number of matches at a position approximately 30 bases upstream of the TSS. 
This suggests the presence of a specific regulatory element that is recognized by TBP at that position.

- what choice(s) in the analysis affect the shape of this distribution?

(Don't be afraid to try out your ideas in separate code cells, but please clean up dead code!)

```
The following are the choices that may affect the shape of distribution:
 1. TFBS motif
 2. gene set
 3. genomic region to analyze
 4. motif match cutoff
 5. smoothing method (e.g., spline interpolation)
```


- define a null and alternate hypothesis for evaluating the "spatial" randomness of matches to a TFB

*By spatial I mean the position along the sequence relative to the TSS.*

Null hypothesis: The occurrence of matches to a TFB is a random event and is not influenced by the spatial position along the sequence relative to the TSS.

Alternative hypothesis: The occurrence of matches to a TFB is not a random event and is influenced by the spatial position along the sequence relative to the TSS.

- describe in pseudo-code how you would implement this

(Pseudo-code is like writing an algorithm in Python, but it contains more English and does not have to be executable. You also do not need to declare data types.)

**NOTE:** do not follow the logic assignment style for this answer.

```
   tss = 0
   pssm_scores = load_pssm_scores(tfbs)
    # Calculate the observed number of matches in each position relative to the TSS
   observed_matches = calculate_matches(pssm_scores, cutoff, tss)
    # Define the null hypothesis
   null_hypothesis = "Matches are randomly distributed across the sequence relative to the TSS"
    # Define the alternate hypothesis
   alternate_hypothesis = "Matches are not randomly distributed across the sequence relative to the TSS"


    # Use a permutation test to evaluate the null hypothesis
   permutation_matches = []
   for i in range(n_permutations)
       # Shuffle the positions relative to the TSS and recalculate the matches
       shuffled_matches = calculate_matches(pssm_scores, cutoff, shuffle_positions(tss))
       permutation_matches.append(shuffled_matches)
    
   p_value = calculate_p_value(observed_matches, permutation_matches)

   # If the p-value is less than the significance threshold, reject the null hypothesis and accept the alternate hypothesis
   if p_value < significance_threshold:
       print("Reject the null hypothesis:", alternate_hypothesis)
   else
        print("Accept the null hypothesis:", null_hypothesis)
```

- are all "matches" to the TBP PSSM likely to be functional? Justify your answer in < 50 words.

No, not all matches to the TBP PSSM are likely to be functional. Some matches may occur by chance and not actually be functional TFBSs. Additionally, the presence of a PSSM match does not guarantee that the TF actually binds to that site in vivo. Functional validation is necessary to determine if a TFBS is actually functional.

## Q1.1 -- optional extension question

Worth 0.5%

Implement in code a method for establishing a suitable cutoff.

In [39]:
import numpy as np
def find_cutoff(pwm_scores):
    cutoff = np.mean(pwm_scores) + np.std(pwm_scores)
    return cutoff

## Q1.2 -- optional extension question

Worth 0.5%

Implement in code your proposed method for testing for spatial randomness.

In [ ]:
import numpy as np

pssm_scores = np.load("data/pssm_scores/tbp.npy")
tss = pssm_scores.shape[0] // 2
window_size = 100
cut_off = find_cutoff(pssm_scores)
observed_matches = np.sum(pssm_scores[tss - window_size:tss + window_size + 1] > cut_off)
n_randomizations = 1000
n_positions = pssm_scores.shape[0]
random_matches = np.zeros(n_randomizations)
for i in range(n_randomizations):
    shuffled_scores = np.random.permutation(pssm_scores)
    random_matches[i] = np.sum(shuffled_scores[tss - window_size:tss + window_size + 1] > cut_off)

p_value = (np.sum(random_matches >= observed_matches) + 1) / (n_randomizations + 1)

fig = px.histogram(random_matches, nbins=20, labels=dict(x="Number of Random Matches"))
fig.add_vline(x=observed_matches, line_color="red", annotation_text="Observed Matches", 
              annotation_position="top right")
fig.update_layout(title="Distribution of Random Matches", xaxis_title="Number of Matches",
                  yaxis_title="Count", showlegend=False)
fig.show()

alpha = 0.05

if p_value < alpha:
    print(f"The p-value ({p_value:.3f}) is less than the significance level ({alpha}),"
          " rejecting the null hypothesis of spatial randomness.")
else:
    print(f"The p-value ({p_value:.3f}) is greater than the significance level ({alpha}),"
          " failing to reject the null hypothesis of spatial randomness.")

## Q2 -- analysis of other TFBS

Modify the code I presented for TBP and apply it to all the TFBS (including TBP). (Feel free to write functions etc.. to simplify your code.)

You can either plot each TFBS result in a single cell, or combine the results into one cell.

### NOTE

You will not score well if:

- you do not change the plot titles
- you do not remove all print statements
- you do not delete all commented out code
- you display the array data

You **will** score well if:

- you write a function that eliminates redundant code

In [ ]:
import plotly.express as px
import numpy as np
def load_scores(array_file):
    pssm_scores = np.load("data/pssm_scores/{}".format(array_file))
    cutoff = find_cutoff(pssm_scores)
    scores = (pssm_scores > cutoff).sum(axis=0)
    TSS = scores.shape[0] // 2
    x_axis = [i - TSS for i in range(scores.shape[0])]
    return scores
    

In [ ]:
def load_xaxis(array_file):
    scores=load_scores(array_file)
    x_axis = [i - TSS for i in range(scores.shape[0])]
    return x_axis

In [ ]:
def plot_matches(scores, x_axis, title):
    if (len(x_axis)>len(scores)):
        x_axis=x_axis[0:len(scores)]
    else:
        scores=scores[0:len(x_axis)]
        
    fig = px.line(
        x=x_axis,
        y=scores,
        line_shape="spline",        
        title=f"{title} Binding Protein PSSM matches",
        labels=dict(x="Position relative to TSS", 
        y="Number of Genes Matching"),
    )
    fig.show()

In [ ]:
for bp in ['gata4.npy', 'mef2.npy', 'myog.npy', 'nkx2-5.npy', 'srf1.npy', 'tbp.npy', 'tbx5.npy']:
    plot_matches(load_scores(bp),load_xaxis(bp), bp.split('.')[0].upper())

## Q3

What were the factors affecting your choices regarding the cutoff(s) you chose?

*Answer in less than 200 words.*

1. The choice of cutoff value for identifying TFBS matches can significantly affect the outcome of the analysis.
2. Setting a low cutoff will increase the sensitivity of the search, but may also increase the number of false positives. 
3. On the other hand, setting a high cutoff will increase the specificity, but may lead to missing genuine TFBS matches. 
4. Therefore, the choice of the cutoff should balance between sensitivity and specificity based on the research question, the   characteristics of the data, and the prior knowledge about the TFBS.
5. In the provided implementation, the find_cutoff function calculates the cutoff value as the mean of the PWM scores plus one standard deviation. This is a common practice for selecting a cutoff value based on the assumption that the scores are distributed normally.

In ≤ 100 words total (does not have to be in logic assignment style)

- Summarise the pattern(s) you observed for the TFBS (don't just list them, are there common patterns, or not).
- For each provided PSSM, is a "match" sufficiently informative to uniquely identify a functioning TFBS? (Justify your answer with reference to the corresponding plot.)


1. From the analysis, the patterns for the different TFBS PSSMs is varying. Some of the PSSMs showed a clear spatial pattern in the occurrence of matches, while others showed no clear pattern and, the number of matches also varied between the different TFBS PSSMs.<br>

2. For some PSSMs, a match appears to be informative enough to identify a functioning TFBS. The TBP PSSM showed a clear spatial pattern in the occurrence of matches, and the number of matches was low, suggesting that the matches are likely to be functional.
3. However, for other PSSMs such as the MEF2 and MYOG PSSMs, the matches were distributed across the sequence and showed no clear spatial pattern. In these cases, a match alone may not be sufficient to identify a functional TFBS.

- across the TFBS, what do the key patterns imply about which predicted occurrences of a motif are likely to actually be functional

*Answer in less than 100 words.*

The key patterns imply that the functional occurrences of the motif are more likely to cluster together near the TSS, rather than being randomly distributed throughout the sequence. This suggests that the regulatory elements controlling gene expression are more likely to be located near the TSS, and that not all predicted occurrences of a motif are likely to be functional.

- do all TFBS have a clear pattern? If not, conjecture why.

*Answer in less than 100 words.*

No, not every TFBS exhibits a distinct pattern. This might be caused by a number of things, like the existence of several overlapping TFBS in the same area or the impact of other regulatory components that affect TFBS occupancy. A variance in the geographical distribution of projected binding sites may result from the fact that various TFs may work in various ways and bind to DNA in various ways.

## Q4

With reference to [Culley et al](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3684448/), identify which of the provided TFBS are likely to be associated with cardiac development.

*Answer in less than 100 words.*

The transcription factors GATA4, MEF2C, and NKX2-5 are significant controllers of heart development, claim Culley et al. Due to the fact that all three of these TFs are represented in the given PSSMs, it is likely that matches to their motifs are connected to cardiac development.

Drawing on your answers above, make a prediction about the distribution of TFBS combining in a cis-regulatory module to affect gene expression. (I suggest picking two known be involved in heart development.)

Your predictions must be something that can be computationally tested using the data in `GSM522308-edited.tsv`.

*Answer in less than 300 words.*

1. Based on the analysis above, it can be predicted that the distribution of TFBS in a cis-regulatory module involved in heart development will not be random but rather show specific patterns in terms of spatial distribution and the number of matches to a given PSSM. 
2. For instance, the TBP PSSM showed a peak in matches around the TSS, which could suggest that functional binding sites tend to occur near the transcription start site. Similarly, the GATA4 PSSM had a bimodal distribution with peaks around -200 and +100 bp relative to the TSS, which could suggest that functional binding sites tend to occur at specific distances from the TSS.

3. To test this prediction using the data in GSM522308-edited.tsv, we could identify genes that are known to be involved in heart development and analyze the distribution of TFBS within their cis-regulatory modules. For example, we could focus on the Nkx2-5 and Gata4 genes, both of which are known to play critical roles in heart development. We could extract the sequences of their cis-regulatory modules from the genome and scan them for matches to the Nkx2-5 and Gata4 PSSMs. 

4. By analyzing the distribution of matches across the cis-regulatory modules, we could test the hypothesis that functional binding sites tend to occur at specific distances from the TSS and show a non-random distribution of matches. If the results support this hypothesis, it could suggest that the spatial distribution of TFBS is an important factor in determining the function of cis-regulatory modules in heart development.

## Q5  -- optional extension question

Worth 1%

In ≤ 200 words, describe an algorithm, in pseudocode, for testing your hypothesis.

1. Load the PWM files for the two TFBS of interest.
2. Load the expression data for the genes of interest (GSM522308-edited.tsv).
3. For each gene:
    a. Determine the distance of each TFBS from the TSS of the gene.
    b. For each distance, count the number of matches to the PWM above a certain cutoff.
    c. Calculate the sum of the counts for each distance for the two TFBS.
    d. If the sum of the counts is above a certain threshold, mark the gene as a potential target.
4. Calculate the co-occurrence of the potential target genes and the two TFBS of interest.
5. Perform a statistical test to determine if the co-occurrence is greater than expected by chance.
6. If the co-occurrence is significant, conclude that the two TFBS are likely to combine in a cis-regulatory module to affect gene expression in heart development.